# 04: Model Usage Demonstration (Adapted for SVM)

This notebook, adapted from Matthew Vella's initial version, demonstrates how to load and use the serialized Maltese sentiment analysis pipeline (which now utilizes an SVM model). It shows how to perform inference on raw text inputs. For successful operation, the `preprocessor.py` module, containing necessary custom class definitions, must be available in the Python import path. The notebook illustrates making predictions and retrieving sentiment scores.

Essential Python libraries (`os`, `sys`, `joblib`, `traceback`) are imported, along with the custom `preprocessor` module.

In [1]:
import os
import sys
import joblib
import traceback

import preprocessor # Needed to deserialize and use the model

This section loads the serialized sentiment analysis pipeline (now containing the SVM model) from its `.joblib` file.

In [2]:
# Loading the pipeline
MODEL_FILENAME = "svm_maltese_sentiment_analyzer.joblib" 
loaded_model_pipeline = None

if os.path.exists(MODEL_FILENAME):
    print(f"Loading model from: {MODEL_FILENAME}")
    try:
        loaded_model_pipeline = joblib.load(MODEL_FILENAME)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"CRITICAL ERROR loading the model: {e}")
        traceback.print_exc() 
        print(f"Ensure dependency `preprocessor.py` is loaded")
        sys.exit(1) # Exit if model cannot be loaded
else:
    print(f"Error: Model file '{MODEL_FILENAME}' not found.")
    print("Please ensure the model was saved correctly using the 'Train and Save' script.")
    sys.exit(1) # Exit if model file not found

Loading model from: svm_maltese_sentiment_analyzer.joblib
Model loaded successfully.


The `get_sentiment_prediction` function uses the loaded pipeline to process raw text, apply necessary preprocessing, and predict the sentiment using the integrated SVM model. It also retrieves prediction probabilities. The function then prints the input text, its preprocessed form, and the final sentiment prediction. Its usage is demonstrated with various example sentences.

In [3]:
# Example Usage
def get_sentiment_prediction(raw_text_input, pipeline):
    try:
        # Step 1: Access individual steps
        custom_preprocessor = pipeline.named_steps['custom_maltese_preprocessor']
        model = pipeline.named_steps['sentiment_model']
        
        # Step 2: Get intermediate preprocessed output
        intermediate_output = custom_preprocessor.transform([raw_text_input])
        preprocessed_text = intermediate_output.iloc[0]
        
        print(f"Input: '{raw_text_input}'")
        print(f"Preprocessed Text: '{preprocessed_text}'")

        # Step 3: Run sentiment model on intermediate output
        prediction_label = model.predict(intermediate_output)[0]
        # SVC has probability=True set in Notebook 02 for predict_proba to work
        probabilities = model.predict_proba(intermediate_output)[0]

        # Step 4: Print output
        print(f"Sentiment: {'Positive' if int(prediction_label) == 1 else 'Negative'} ({prediction_label})")
        print(f"Probability (Negative): {float(probabilities[0]):.4f}")
        print(f"Probability (Positive): {float(probabilities[1]):.4f}")

    except Exception as e:
        print(f"Error during prediction for '{raw_text_input}': {e}")


# Example Usage
sentences_to_predict_list = [
    "Dan il-film kien assolutament eccelenti!",
    "Dan is-servizz kien verament tajjeb u għoġobni ħafna!",
    "Xejn speċjali, ma tantx impressjonani.",
    "Xejn ma ghogobni",
    "This is an English sentence."
    "Tal misthija",
    "Qazziztni",
    "Ma nifilhekx",
    "Tal-Ostja Int u t team kollhu tieghek!",
    "Inkredibbli kif irnexxielhom jkisruha diska.",
    "Prosit ministru",
    "Inkredibbli", # Incorrectly labelled as Negative
    "Kilt ikel tajjeb", # Incorrectly labelled as Negative

    "nisrani", # Positive?
    "musulman", # Negative?
    ]

print("\nPredicting for a list of sentences:")
for sentence in sentences_to_predict_list:
    result = get_sentiment_prediction(sentence, loaded_model_pipeline)
    print()


Predicting for a list of sentences:
Input: 'Dan il-film kien assolutament eccelenti!'
Preprocessed Text: 'dan il film kien assolutament eccelenti !'
Sentiment: Negative (0)
Probability (Negative): 0.7024
Probability (Positive): 0.2976

Input: 'Dan is-servizz kien verament tajjeb u għoġobni ħafna!'
Preprocessed Text: 'dan is servizz kien verament tajjeb u għoġob ħaf !'
Sentiment: Positive (1)
Probability (Negative): 0.0241
Probability (Positive): 0.9759

Input: 'Xejn speċjali, ma tantx impressjonani.'
Preprocessed Text: 'xejn speċjali , ma tantx impressjonani .'
Sentiment: Negative (0)
Probability (Negative): 0.8936
Probability (Positive): 0.1064

Input: 'Xejn ma ghogobni'
Preprocessed Text: 'xejn ma ghogobni'
Sentiment: Negative (0)
Probability (Negative): 0.9876
Probability (Positive): 0.0124

Input: 'This is an English sentence.Tal misthija'
Preprocessed Text: 'this is an english sentence . tal mistħi'
Sentiment: Negative (0)
Probability (Negative): 0.9563
Probability (Positive): 0.